# Validation

In order to validate our results, this notebook will show the flood mapping for the same area and time range from the ['Global Flood Monitoring'(GFM)](https://services.eodc.eu/browser/#/v1/collections/GFM) to allow comparisons.

In [ ]:
import hvplot.xarray
import pystac_client
from odc import stac as odc_stac
from dask.distributed import Client, wait

#remove client afterwards
client = Client(processes=False, threads_per_worker=2, n_workers=3, memory_limit="12GB")

# Connect to STAC catalog
eodc_catalog = pystac_client.Client.open("https://stac.eodc.eu/api/v1/")

# Search for available images
time_range = "2023-10-11/2023-10-25"
bounding_box = [12.3, 54.3, 13.1, 54.6]
search = eodc_catalog.search(collections="GFM", bbox=bounding_box, datetime=time_range)
items_GFM = search.item_collection()

print(f"Found {len(items_GFM)} items") 

### Load the data

In [ ]:
import pyproj

crs = "EPSG:4326"  # Coordinate Reference System - World Geodetic System 1984 (WGS84) in this case
resolution = 0.00018  # 20 meter in degree

GFM_dc= odc_stac.load(
    items_GFM, 
    bbox=bounding_box,   
    crs=crs,  
    bands=["tuw_likelihood","tuw_flood_extent"],   
    resolution=resolution,   
    dtype='uint8',   
    chunks={"x": 1000, "y": 1000, "time": -1},  
)



### Pre-processing 

In [ ]:
GFM_dc["tuw_flood_extent"] = GFM_dc.tuw_flood_extent.where(GFM_dc.tuw_flood_extent!=255).compute()

GFM_dc_flood = GFM_dc["tuw_flood_extent"]

#GFM_dc_flood.data

In [ ]:
import numpy as np

values = GFM_dc_flood.values
unique_values, counts = np.unique(values[~np.isnan(values)], return_counts=True)
print(dict(zip(unique_values, counts)))


### Plot GFM



In [ ]:
from bokeh.models import FixedTicker
import hvplot.xarray

band_name = "tuw_flood_extent"
print(f"Visualizing band: {band_name}")

colorbar_opts = {
    "major_label_overrides": {
        0: "non-flood",
        1: "flood",
    },
    "ticker": FixedTicker(ticks=[0, 1]),
}

GFM_flood_plot =GFM_dc_flood.hvplot.image(x="longitude", y="latitude", title=band_name, cmap=["rgba(0, 0, 1, 0.1)", "darkred"]).opts(frame_height=400, frame_width= 700)
GFM_flood_plot



In [ ]:
GFM_plot=GFM_dc_flood.hvplot.quadmesh(
    x="longitude",
    y="latitude",
  #  rasterize=True,
   # geo=True,
    cmap=["rgba(0, 0, 1, 0.1)", "darkred"],
).opts(frame_height=400, frame_width= 700)

GFM_plot

## Plot dask-flood-mapper

In [ ]:
from dask_flood_mapper import flood
from dask.distributed import Client

# define dask client
client = Client(processes=False, threads_per_worker=2, n_workers=1, memory_limit="12GB")

# flood area and time
time_range = "2023-10-11/2023-10-25"
bbox = [12.3, 54.3, 13.1, 54.6]
#calculate flood decision
fd = flood.decision(bbox=bbox, datetime=time_range).compute()

from importlib.resources import files
import xarray as xr

data_text = files("dask_flood_mapper.data").joinpath("wcover.tif")
wcover = xr.open_dataarray(data_text)

from bokeh.models import FixedTicker
import hvplot.xarray



In [ ]:
fd = fd.where(wcover != 80)
colorbar_opts = {
    "major_label_overrides": {
        0: "non-flood",
        1: "flood",
    },
    "ticker": FixedTicker(ticks=[0, 1]),
}
plot_obj=fd.hvplot.quadmesh(
    x="x",
    y="y",
    rasterize=True,
    geo=True,
    cmap=["rgba(0, 0, 1, 0.1)", "darkred"],
).opts(frame_height=400, frame_width= 700)#, colorbar_opts={**colorbar_opts})

plot_obj

In [ ]:
combined_plot = GFM_flood_plot + plot_obj
#combined_plot.opts(
 #   title="Comparison of Flood Extent and Water Cover",height=800,width=400)
combined_plot

In [ ]:
fd

In [ ]:
import numpy as np
import holoviews as hv
import hvplot.xarray

hv.extension('bokeh')

fd = fd.rename({'x':'longitude', 'y':'latitude'})
# Get the shared time values (intersection) between the two datasets
common_times = np.intersect1d(GFM_dc_flood.time.values, fd.time.values)

# Define the plot function for the synced time slider
def synced_plot(t):
    # Slice the datasets by the common time values (ignoring 'band')
    plot1 = GFM_dc_flood.sel(time=t).hvplot.image(x='longitude', y='latitude')
    plot2 = fd.sel(time=t).hvplot.image(x='longitude', y='latitude')

    # Return both plots side by side
    return (plot1 + plot2).cols(2)

# Create the DynamicMap with the common time range
combined_plot = hv.DynamicMap(synced_plot, kdims='time').redim.values(time=common_times)

combined_plot
